In [25]:
import gym
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [38]:
class lstmPolicyPredictor(nn.Module):

    def __init__(self, inp_dim, hidden_dim, out_hidden_dim):
        super(lstmPolicyPredictor, self).__init__()
        #self.hidden_dim = hidden_dim


        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(inp_dim, hidden_dim)
        self.out1 = nn.Linear(hidden_dim+1, out_hidden_dim)
        self.out2 = nn.Linear(out_hidden_dim, 1)


    def forward(self, inp, oldstate=None):
        allinp = inp#torch.cat((inp, lastaction))
        lstmout, newstate = self.lstm(allinp.view(1,1,-1), oldstate)
        allforout = torch.cat((lstmout.view(-1), inp))
        h1 = F.relu(self.out1(allforout))
        out = F.sigmoid(self.out2(h1))
        return out, newstate

In [68]:
model = lstmPolicyPredictor(1,8,16)

In [17]:
testt = torch.tensor([7,8], dtype=torch.float).view(1,1,-1)
testi = torch.tensor([7], dtype=torch.float)

In [14]:
pp.lstm(testt)[1]

(tensor([[[-0.3243, -0.0051, -0.0053]]], grad_fn=<StackBackward>),
 tensor([[[-0.3423, -0.2773, -0.1022]]], grad_fn=<StackBackward>))

In [28]:
data = torch.empty([1024, 1024], dtype=torch.float)

In [29]:
for i in range(1024):
    for k in range(1024):
        data[i][k] = 2.0 if pol2.getpixel((i,k)) > 200 else 0.0

In [39]:
import bz2
import gzip
import pickle
#bz2.BZ2File
sfile = gzip.open('lookup_policy', 'wb')
#pickle.dump(data, sfile)
torch.save(data, sfile)
sfile.close()

In [40]:
data

tensor([[0., 0., 0.,  ..., 2., 2., 2.],
        [0., 0., 0.,  ..., 2., 2., 2.],
        [0., 0., 0.,  ..., 2., 2., 2.],
        ...,
        [0., 0., 0.,  ..., 2., 2., 2.],
        [0., 0., 0.,  ..., 2., 2., 2.],
        [0., 0., 0.,  ..., 2., 2., 2.]])

In [45]:
rfile = gzip.open('lookup_policy', 'rb')
torch.load(rfile)

tensor([[0., 0., 0.,  ..., 2., 2., 2.],
        [0., 0., 0.,  ..., 2., 2., 2.],
        [0., 0., 0.,  ..., 2., 2., 2.],
        ...,
        [0., 0., 0.,  ..., 2., 2., 2.],
        [0., 0., 0.,  ..., 2., 2., 2.],
        [0., 0., 0.,  ..., 2., 2., 2.]])

In [27]:
env = gym.make('MountainCar-v0')

/opt/conda/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [27]:
def dopol(obs, pol):
    px = int((obs[0]-env.min_position)/(env.max_position - env.min_position) * 1023.9999)
    py = int((obs[1]+env.max_speed)/(2*env.max_speed) * 1023.9999)
    return 1 if pol.getpixel((px,py)) > 200 else 0
pol2 = Image.open('policy2.png').convert('L')

In [80]:
bce = nn.BCELoss(reduction='sum')
optimizer = optim.Adam(model.parameters(), lr=0.00001)

In [107]:
for i in range(1000):
    model.zero_grad()
    observation = env.reset()
    state = None
    outputs = torch.tensor([], dtype=torch.float)
    targets = torch.tensor([], dtype=torch.float)
    for t in range(300):
        output, state = model(torch.tensor(observation[:1], dtype=torch.float), state)
        action = 2 if output[0] > 0.5 else 0#0 if t==0 else 2 #2 #0 if not tree1.get(observation) and tree.get(observation) else 2
        outputs = torch.cat((outputs, output))
        targets = torch.cat((targets, torch.tensor([dopol(observation, pol2)], dtype=torch.float)))
        #print(action, dopol(observation, pol2)*2, loss)
        #env.render()
        observation, _, done, _ = env.step(action)
        if done:
            loss = bce(outputs, targets)
            loss.backward()
            optimizer.step()
            if i%10 == 0:
                print(loss, ((outputs>0.5) != targets).sum(), t+1)
                #print("Episode finished after {} timesteps".format(t+1))
            break
    

tensor(7.4946, grad_fn=<BinaryCrossEntropyBackward>) tensor(2) 103
tensor(6.3647, grad_fn=<BinaryCrossEntropyBackward>) tensor(1) 83
tensor(7.6860, grad_fn=<BinaryCrossEntropyBackward>) tensor(2) 104
tensor(7.8486, grad_fn=<BinaryCrossEntropyBackward>) tensor(2) 104
tensor(7.9287, grad_fn=<BinaryCrossEntropyBackward>) tensor(2) 88
tensor(8.1993, grad_fn=<BinaryCrossEntropyBackward>) tensor(2) 103
tensor(7.6771, grad_fn=<BinaryCrossEntropyBackward>) tensor(0) 103
tensor(7.5803, grad_fn=<BinaryCrossEntropyBackward>) tensor(2) 87
tensor(13.2419, grad_fn=<BinaryCrossEntropyBackward>) tensor(3) 91
tensor(8.1138, grad_fn=<BinaryCrossEntropyBackward>) tensor(2) 89
tensor(8.3816, grad_fn=<BinaryCrossEntropyBackward>) tensor(2) 103
tensor(23.7134, grad_fn=<BinaryCrossEntropyBackward>) tensor(11) 125
tensor(7.8473, grad_fn=<BinaryCrossEntropyBackward>) tensor(0) 103
tensor(10.4222, grad_fn=<BinaryCrossEntropyBackward>) tensor(2) 103
tensor(21.4379, grad_fn=<BinaryCrossEntropyBackward>) tensor(6)

In [108]:
(outputs>0.5) != targets

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False,  True, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False,  True, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False])

In [96]:
torch.tensor([1,2])==torch.tensor([0,2])

tensor([False,  True])